## Imports

In [ ]:
from __future__ import print_function, division

from keras.datasets import mnist, fashion_mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

from matplotlib.image import imread
import cv2
import os
from PIL import Image

import sys

import numpy as np

## Methods

In [48]:
# method for loading all the images from a folder
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = rgb2gray(cv2.imread(os.path.join(folder,filename)))
        if img is not None:
            images.append(img)
    return images

In [ ]:
def save_images_to_folder(folder, images):
    img_cnt = 1
    for img in images:
        cv2.imwrite(os.path.join(folder , '{}.jpg'.format(img_cnt)), img)
        img_cnt += 1
#         print(os.path.join(folder , '{}.jpg'.format(img_cnt)))

In [ ]:
# method for loading all the images from a folder
def imread_load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images

In [49]:
def rgb2gray(rgb):

    r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b

    return gray

## Pre-processing

In [ ]:
# original images path
org_norm_path = './000444/without'
org_occ_path = './000444/with_occlussion'

# set the path for storing the converted images
arr_norm_path = './numpy_img/normal'
arr_occ_path = './numpy_img/occluded'

# convert the images into an array
norm_images = load_images_from_folder(org_norm_path)
save_images_to_folder(arr_norm_path, norm_images)

occ_images = load_images_from_folder(org_occ_path)
save_images_to_folder(arr_occ_path, occ_images)

In [54]:
(X_train, _), (_, _) = mnist.load_data()
X_train.shape

(60000, 28, 28)

In [56]:
norm_images = load_images_from_folder(org_norm_path)
np.asarray(norm_images).shape

(28, 600, 600)

## GAN

In [59]:
class GAN():
    def __init__(self):
        self.img_rows = 600
        self.img_cols = 600
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)


    def build_generator(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)
    
    @timeit
    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
#         (X_train, _), (_, _) = mnist.load_data()
        norm_images = load_images_from_folder(org_norm_path)
        X_train = np.asarray(norm_images)
        
        '''
            Why expand dimensions to 3?
        '''
        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as valid)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/%d.png" % epoch)
        plt.close()


if __name__ == '__main__':
    gan = GAN()
    gan.train(epochs=20, batch_size=4, sample_interval=4)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 360000)            0         
_________________________________________________________________
dense_8 (Dense)              (None, 512)               184320512 
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 257       
Total params: 184,452,097
Trainable params: 184,452,097
Non-trainable params: 0
______________________________________________________________

/home/critin/miniconda3/envs/ml_werl/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.487303, acc.: 75.00%] [G loss: 0.635247]


/home/critin/miniconda3/envs/ml_werl/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [D loss: 0.198529, acc.: 87.50%] [G loss: 2.408710]
2 [D loss: 0.000000, acc.: 100.00%] [G loss: 4.134848]
3 [D loss: 1.992806, acc.: 87.50%] [G loss: 8.059048]
4 [D loss: 4.942045, acc.: 62.50%] [G loss: 4.058783]
5 [D loss: 2.711708, acc.: 75.00%] [G loss: 8.077259]
6 [D loss: 1.992800, acc.: 87.50%] [G loss: 8.059048]
7 [D loss: 3.985596, acc.: 75.00%] [G loss: 4.176013]
8 [D loss: 1.256688, acc.: 87.50%] [G loss: 12.088572]
9 [D loss: 3.985868, acc.: 75.00%] [G loss: 8.059048]
10 [D loss: 1.992798, acc.: 87.50%] [G loss: 5.947866]
11 [D loss: 1.992798, acc.: 87.50%] [G loss: 8.059048]
12 [D loss: 2.035523, acc.: 87.50%] [G loss: 5.720737]
13 [D loss: 3.985596, acc.: 75.00%] [G loss: 5.181320]
14 [D loss: 3.985596, acc.: 75.00%] [G loss: 4.029524]
15 [D loss: 7.971192, acc.: 50.00%] [G loss: 4.029524]
16 [D loss: 3.985596, acc.: 75.00%] [G loss: 8.059048]
17 [D loss: 5.978395, acc.: 62.50%] [G loss: 4.029524]
18 [D loss: 3.985596, acc.: 75.00%] [G loss: 8.059048]
19 [D loss: 1.992

In [58]:
import time
from functools import wraps
def timeit(f):
    @wraps(f)
    def wrapper(*args, **kwds):
        start_time = time.time()
        result = f(*args, **kwds)
        elapsed_time = time.time() - start_time
        print('Elapsed computation time: {:.3f} secs'.format(elapsed_time))
        return elapsed_time, result
    return wrapper